## Plot point clouds with maps

In [1]:
from typing import List

import numpy as np
import plotly.graph_objs as go
from waymo_open_dataset import dataset_pb2
from waymo_open_dataset.utils import frame_utils
from waymo_open_dataset.utils import plot_maps

def plot_point_clouds_with_maps(frames: List[dataset_pb2.Frame])->None:
  """Plot the point clouds within the given frames with map data.

  Map data must be populated in the first frame in the list.

  Args:
    frames: A list of frames to be plotted, frames[0] must contain map data.
  """

  # Plot the map features.
  if len(frames) == 0:
    return
  figure = plot_maps.plot_map_features(frames[0].map_features)

  for frame in frames:
    # Parse the frame lidar data into range images.
    range_images, camera_projections, seg_labels, range_image_top_poses = (
        frame_utils.parse_range_image_and_camera_projection(frame)
    )

    # Project the range images into points.
    points, cp_points = frame_utils.convert_range_image_to_point_cloud(
        frame,
        range_images,
        camera_projections,
        range_image_top_poses,
        keep_polar_features=True,
    )
    xyz = points[0][:, 3:]
    num_points = xyz.shape[0]
    
    # Transform the points from the vehicle frame to the world frame.
    xyz = np.concatenate([xyz, np.ones([num_points, 1])], axis=-1)
    transform = np.reshape(np.array(frame.pose.transform), [4, 4])
    xyz = np.transpose(np.matmul(transform, np.transpose(xyz)))[:, 0:3]

    # Correct the pose of the points into the coordinate system of the first
    # frame to align with the map data.
    offset = frame.map_pose_offset
    points_offset = np.array([offset.x, offset.y, offset.z])
    xyz += points_offset

    # Plot the point cloud for this frame aligned with the map data.
    intensity = points[0][:, 0]
    figure.add_trace(
        go.Scatter3d(
            x=xyz[:, 0],
            y=xyz[:, 1],
            z=xyz[:, 2],
            mode='markers',
            marker=dict(
                size=1,
                color=intensity,  # set color to an array/list of desired values
                colorscale='Pinkyl',  # choose a colorscale
                opacity=0.8,
            ),
        )
    )

  figure.show()


2023-03-23 18:41:26.609244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 18:41:27.087125: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/k/.mujoco/mujoco200/bin
2023-03-23 18:41:27.087188: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/k/.mujoco/mujoco200/bin
2023-03-23 18:41:27.087194: W tensorflow/compiler/tf2tensorrt/utils/

## Load frames data

In [3]:
import tensorflow as tf

FILENAME = '/efs/waymo_open_with_maps/training/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord'

dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')

# Load only 2 frames. Note that using too many frames may be slow to display.
frames = []
count = 0
for data in dataset:
  frame = dataset_pb2.Frame.FromString(bytearray(data.numpy()))
  frames.append(frame)
  count += 1
  if count == 2: 
    break


2023-03-23 18:41:28.778289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-23 18:41:28.778587: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-23 18:41:28.778855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-23 18:41:28.779123: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-23 18:41:28.784104: W tensorflow/compiler/xla/stream_executo

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} /content/frames_with_maps.tfrecord; No such file or directory [Op:IteratorGetNext]

## Plot frames data

In [ ]:
# Interactive plot of multiple point clouds aligned to the maps frame.

# For most systems:
#   left mouse button:   rotate
#   right mouse button:  pan
#   scroll wheel:        zoom

plot_point_clouds_with_maps(frames)